In [1]:
import numpy as np
import random
import pandas as pd
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from matplotlib import dates
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime
from lmfit import minimize, Parameters, Parameter, report_fit
from statsmodels.tsa.arima_model import ARIMA

# step 0: data importation #

In [2]:
csv_url="https://raw.githubusercontent.com/ADelau/proj0016-epidemic-data/main/data.csv"
data = pd.read_csv(csv_url)

In [3]:
data.head(5)

,Day,num_positive,num_tested,num_hospitalised,num_cumulative_hospitalizations,num_critical,num_fatalities
0,1,0,0,1,1,0,0
1,2,5,8,1,1,0,0
2,3,10,12,2,2,0,0
3,4,11,16,2,2,0,0
4,5,9,12,2,2,0,0


# step 1: setting up the model #

## step 1.1: writing the ode system ##

In [49]:
def deriv(y,t,ps):
    N=10e6
    S,E,I,C,R,H,ICU,D = y
    try:
        beta_SE = ps['beta_SE'].value
        gamma_ER = ps['gamma_ER'].value
        fraction_symptomatic = ps['fraction_symptomatic'].value
        days_EtoI=ps['days_EtoI'].value
        days_ItoH=ps['days_ItoH'].value
        days_ItoR=ps['days_ItoR'].value
        days_HtoR=ps['days_HtoR'].value
        days_HtoICU=ps['days_HtoICU'].value
        days_ICUtoD=ps['days_ICUtoD'].value
        days_ICUtoH=ps['days_ICUtoH'].value
        
    except:
        beta_SE, gamma_ER, fraction_symptomatic, days_EtoI, days_ItoH, days_ItoR, days_HtoR, days_HtoICU, days_ICUtoD, days_ICUtoH= ps
    
    #beta = beta_i*(1.1-tau*t) idée de faire régresser beta
    dSdt = -beta_SE*(S*(I+C))/(N-D)
    dEdt = beta_SE*(S*(I+C))/(N-D)-gamma_ER*E
    dIdt = (fraction_symptomatic)*E/days_EtoI-I*(1/days_ItoH+1/days_ItoR)
    dCdt = (1-fraction_symptomatic)*E/days_EtoI-C/days_ItoR
    dRdt = (C+I)/days_ItoR+H/days_HtoR
    dHdt = I/days_ItoH*I+ICU/days_ICUtoH-H*(1/days_HtoICU+1/days_HtoR)
    dICUdt = H/days_HtoICU-ICU*(1/days_ICUtoD+1/days_ICUtoH)
    dDdt = ICU/days_ICUtoD
    
    return dSdt, dEdt, dIdt, dCdt, dRdt, dHdt, dICUdt, dDdt

## step 1.2: writing the parameters values and guesses ##


In [50]:
# the lmfit module uises an orderd dict structure
# to store the parameters to be optimized
# https://lmfit.github.io/lmfit-py/parameters.html

params = Parameters()
params.add('beta_SE', value=0.15, min=0.001, max=2)
params.add('gamma_ER', value= 0.05, min=0.1, max=2)
params.add('fraction_symptomatic', value= 0.6, min=0.5, max=0.7)
params.add('days_EtoI', value= 3, min=1, max=5)
params.add('days_ItoH',value=5, min=1,max=20)
params.add('days_ItoR',value=10,min=1,max=50)
params.add('days_HtoR',value=12,min=1,max=100)
params.add('days_HtoICU',value=5,min=1,max=50)
params.add('days_ICUtoD',value=2,min=1,max=50)
params.add('days_ICUtoH',value=6,min=1,max=50)
params.add('I0',value=5,min=1,max=50)
params.add('E0',value=10,min=1,max=40)
params.add('I0',value=5,min=1,max=20)
params.add('C0',value=5,min=0,max=20)
params.add('OBS_Tr_EI_to_nbTest',value=0.25,min=0.05,max=1)
params.add('OBS_nbTest_to_nbpos',value=0.75,min=0.5,max=0.9)
# params.add('OBS_H_to_ICU',value=0.1,min=0.01,max=0.5) mistake??



## step 1.3 writing the solver ##

In [51]:
N=10e6
E0=params['E0'].value
I0 = params['I0'].value
C0=params['C0'].value
S0=N-I0-E0-C0-1
R0=0
H0=1
ICU0=0
D0=0

y0 = S0,E0,I0,C0,R0,H0,ICU0,D0

In [52]:
np.linspace(0,10,11)

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])

In [68]:
#this function solves the ode
#input: function deriv, initial compartiments y0, t, and N,ps as arguments??
#output: the ode solution
def odesol(y,t,ps):

    #x = odeint(deriv, y0, t, args=(ps))
    ts=np.linspace(0,t,t+1)
    x = odeint(deriv,y,ts,args=(ps,))
    
    return x



In [69]:
x=odesol(y0,1,params)

In [70]:
x

array([[9.99997900e+06, 1.00000000e+01, 5.00000000e+00, 5.00000000e+00,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [9.99997740e+06, 1.05713131e+01, 5.48179152e+00, 5.82839907e+00,
        1.34217303e+00, 5.59843195e+00, 5.21072548e-01, 1.06640650e-01]])

In [56]:
# this function generate the observation dataframe
def create_obsdf(model,ps,data):
    result=pd.DataFrame(columns=data.columns)
    result['Day']=data['Day']
    nb_trans_EI=ps['beta_SE']*(model['S']*(model['I']+model['C']))/(10**6-model['D'])
    result['num_tested']=ps['OBS_Tr_EI_to_nbTest'].value*nb_trans_EI
    result['num_positive']=ps['OBS_nbTest_to_nbpos'].value*result['num_tested']
    result['num_hospitalised']=model['H']
    result['num_cumulative_hospitalizations']=result['num_hospitalised'].diff().fillna(0).cumsum()
    result['num_critical']=model['ICU']
    result['num_fatalities']=model['D']
    return result
    

#test['num_tested']=params['OBS_Tr_EI_to_nbTest'].value*

# step 2: fitting the model #

## step 2.1: write objective function ##

In [75]:
#this function compute residuals,
#ie objective function to be minimized in the optimization function
def residual(ps, data):
    N=10e6
    E0=ps['E0'].value
    I0 = ps['I0'].value
    C0=ps['C0'].value
    S0=N-I0-E0-C0-1
    R0=0
    H0=1
    ICU0=0
    D0=0
    
    y0 = S0,E0,I0,C0,R0,H0,ICU0,D0
    
    #cette boucle calculer un résidu pour chaque pas de temps
    # ici ce pas =1
    residu=np.ndarray(data.shape[0])
    for i,line in data.iterrows():
        #créer une observation
        temp=odesol(y0,1,ps)
        obs_df=create_obsdf(pd.DataFrame(temp,columns=['S','E','I','C','R','H','ICU','D']),ps,line)
        component1=(obs_df['num_positive']-line['num_positive']).ravel()
        component2=(obs_df['num_tested']-line['num_tested']).ravel()
        component3=(obs_df['num_hospitalised']-line['num_hospitalised']).ravel()
        component4=(obs_df['num_critical']-line['num_critical']).ravel()
        component5=(obs_df['num_fatalities']-line['num_fatalities']).ravel()
        residu[i]=component1
        #generate compartiment data at time t+1
        y0=temp
        
        
        
    
    #model = pd.DataFrame(odesol(y0,t,ps), columns=['S','E','I','C','R','H','ICU','D'])
    #obs_df=create_obsdf(model,ps,data)

    # penalty function to think more about !!!
    return residu

In [74]:
np.ndarray(data.shape[0])

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38.,
       39., 40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51.,
       52., 53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64.,
       65., 66., 67., 68., 69., 70.])

In [60]:
t = np.linspace(0, data.shape[0]-1, data.shape[0])
residual(params,t,data)

TypeError: only integer scalar arrays can be converted to a scalar index

## step 2.2: write optimization command ##

In [16]:
result = minimize(residual, params, args=(t, data), method='leastsq')

C:\Users\DimiP\anaconda3\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


In [ ]:
%%debug
minimize

In [17]:
result

In [260]:
1 - result[''}.residual.var() / np.var()

TypeError: _var_dispatcher() missing 1 required positional argument: 'a'

In [256]:
final = data + result.residual.reshape(data.shape)
# display fitted statistics
report_fit(result)

ValueError: cannot reshape array of size 71 into shape (71,7)